In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 顶点客户端库：用于在线预测的自定义训练表格回归模型，附带说明

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_tabular_regression_online_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_tabular_regression_online_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示如何使用Python的Vertex客户端库来训练和部署一个自定义的表格回归模型，以便进行在线预测并提供解释。

### 数据集

本教程使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。本教程使用的数据集版本已经内置在 TensorFlow 中。训练好的模型预测房屋的中位价格，单位为1K 美元。

### 目标

在本教程中，您将使用Vertex客户端库在Google预构建的Docker容器中的Python脚本中创建自定义模型，然后通过发送数据对部署的模型进行预测并解释结果。您还可以选择使用`gcloud`命令行工具或Google Cloud Console在线创建自定义模型。

执行的步骤包括：

- 创建一个用于训练模型的Vertex自定义作业。
- 训练一个TensorFlow模型。
- 检索并加载模型工件。
- 查看模型评估。
- 设置解释参数。
- 将模型上传为Vertex`Model`资源。
- 将`Model`资源部署到服务`Endpoint`资源。
- 进行带有解释的预测。
- 撤销`Model`资源的部署。

成本

本教程使用 Google 云（GCP）的可计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
定价](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
生成基于您预计使用量的成本估算。

安装

安装最新版的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的 google-cloud-storage 库的 GA 版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

安装本教程所需的其他包。

In [ ]:
! pip3 install -U tabulate $USER_FLAG

### 重启内核

一旦您安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU运行时

*如果有选择的话，请确保你正在使用GPU运行此笔记本。在Colab中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的Google Cloud项目

**无论您使用什么笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建一个帐户时，您将获得300美元的免费信用额用于计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用Vertex API和Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在Google Cloud Notebook中。

5. 在下面的单元格中输入您的项目ID。然后运行单元格以确保
Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行作为shell命令运行，并且会将以`$`为前缀的Python变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改`REGION`变量，该变量用于本笔记本其余部分的操作。以下是Vertex支持的地区。我们建议您选择离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不会在Vertex中使用多地区存储桶进行训练。并非所有地区都支持所有Vertex服务。有关最新每个地区的支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您在进行实时教程会话，您可能会使用一个共享的测试账户或项目。为了避免用户之间在创建的资源上发生名称冲突，您可以为每个实例会话创建一个时间戳，并将其附加到本教程中将要创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

###验证您的谷歌云账户

**如果您正在使用谷歌云笔记本**，您的环境已经通过验证。跳过这一步。

**如果您在使用Colab**，运行下面的单元格，并按照提示进行账户验证以通过oAuth进行验证。

**否则**，请按照以下步骤进行操作：

在云控制台中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**栏中输入名称，然后点击**创建**。

在**将此服务账户授予项目访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，并选择**Vertex管理员**。在过滤框中输入"Storage Object Admin"，并选择**Storage Object Admin**。

点击创建。一个包含您密钥的JSON文件将下载到您的本地环境。

在下面的单元格中将您的服务账户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用什么笔记本环境，以下步骤都是必须的。**

当您使用 Vertex 客户端库提交一个自定义训练作业时，您需要将包含训练代码的 Python 软件包上传到一个云存储桶中。Vertex 会从这个软件包中运行代码。在本教程中，Vertex 还会将您的作业生成的训练模型保存在同一个存储桶中。然后，您可以根据这个输出创建一个`Endpoint`资源，以便提供在线预测。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在时才能运行以下单元格来创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常数

导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

import google.cloud.aiplatform_v1beta1 as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、流水线和端点服务的顶点API服务端点。
- `PARENT`：用于数据集、模型、作业、流水线和端点资源的顶点位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### 硬件加速器

设置硬件加速器（例如，GPU）用于训练和预测。

设置变量 `TRAIN_GPU/TRAIN_NGPU` 和 `DEPLOY_GPU/DEPLOY_NGPU` 使用支持 GPU 的容器映像，并指定分配给虚拟机（VM）实例的 GPU 数量。例如，要使用一个带有 4 个 Nvidia Telsa K80 GPU 的容器映像分配给每个 VM，您需要指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于 GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，请指定 `(None, None)` 以使用一个容器映像在 CPU 上运行。

*注意*：TF 在 2.3 之前的版本中，对 GPU 支持将无法加载这个教程中的自定义模型。这是一个已知问题，在 TF 2.3 中已修复 -- 这是由于在服务函数中生成的静态图操作。如果在自己的自定义模型上遇到这个问题，请使用支持 GPU 的 TF 2.3 的容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）镜像

接下来，我们将为训练和预测设置 Docker 容器镜像

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
- TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
- Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

有关最新列表，请查看[用于训练的预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)。

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

有关最新列表，请查看[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个虚拟 CPU 3.75GB 内存。
     - `n1-highmem`：每个虚拟 CPU 6.5GB 内存。
     - `n1-highcpu`：每个虚拟 CPU 0.9GB 内存。
 - `虚拟 CPU`：数量为 \[2, 4, 8, 16, 32, 64, 96\]

*注意：以下情况不支持训练：*

 - `standard`：2虚拟 CPU
 - `highcpu`：2、4 和 8 虚拟 CPU

*注意：您也可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持 GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在你已经准备好开始为波士顿房屋创建自定义模型和训练。

## 设置客户端

Vertex客户端库作为客户端/服务器模型工作。在您的一侧（Python脚本中），您将创建一个客户端，该客户端向Vertex服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置好它们。

- 为`Model`资源设置模型服务。
- 用于部署的端点服务。
- 用于批处理作业和自定义训练的作业服务。
- 用于提供服务的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

训练模型

有两种方法可以使用容器镜像训练自定义模型：

- **使用谷歌云预构建的容器**。如果使用预构建的容器，你还需要指定一个要安装到容器镜像中的Python包。这个Python包包含了你用来训练自定义模型的代码。

- **使用您自己的自定义容器镜像**。如果使用自己的容器，容器需要包含用于训练自定义模型的代码。

准备您的自定义工作规范

现在您的客户准备好了，您的第一步是为您的自定义培训工作创建一份工作规范。工作规范将包括以下内容：

- `worker_pool_spec`：用于培训的机器类型规范及数量（单个或分布式）
- `python_package_spec`：要与预构建容器一起安装的Python包规范。

### 准备您的机器规格

现在为您的自定义训练作业定义机器规格。这将告诉 Vertex 需要为训练提供什么类型的机器实例。
- `machine_type`：要提供的 GCP 实例的类型，例如 n1-standard-8。
- `accelerator_type`：（如果有的话）硬件加速器的类型。在本教程中，如果您之前设置了变量 `TRAIN_GPU != None`，则使用 GPU；否则将使用 CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在为您的自定义训练作业定义磁盘规格。这将告诉 Vertex 在每台机器实例中为训练提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD 或标准。SSD 更快，标准更便宜。默认为 SSD。
- `boot_disk_size_gb`：磁盘大小，以 GB 为单位。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

###定义工作程序池规范

接下来，您需要为自定义训练任务定义工作程序池规范。工作程序池规范将包括以下内容：

- `replica_count`：要提供的此机器类型的实例数。
- `machine_spec`：硬件规范。
- `disk_spec`：（可选）磁盘存储规范。

- `python_package`：要安装在VM实例上的Python训练包，以及要调用的Python模块，以及Python模块的命令行参数。

现在让我们深入了解Python包规范：

- `executor_image_spec`：这是为您的自定义训练任务配置的Docker镜像。

- `package_uris`：这是要安装在提供的实例上的Python训练包的位置（URI）列表。这些位置需要在Cloud Storage存储桶中。这些可以是单个Python文件或整个包的zip（归档）。在后一种情况下，作业服务将解压缩（解档）内容到Docker镜像中。

- `python_module`：要调用以运行自定义训练任务的Python模块（脚本）。在本例中，您将调用 `trainer.task.py` -- 请注意，没有必要附加 `.py` 后缀。

- `args`：要传递给相应Python模块的命令行参数。在本例中，您将设置：
  - `"--model-dir=" + MODEL_DIR`：存储模型工件的Cloud Storage位置。告诉训练脚本保存模型工件的两种方式有两种：
     - 直接: 将Cloud Storage位置作为命令行参数传递给您的训练脚本（设置变量 `DIRECT = True`）或
     - 非直接: 服务将Cloud Storage位置作为环境变量 `AIP_MODEL_DIR` 传递给您的训练脚本（设置变量 `DIRECT = False`）。在这种情况下，您在作业规范中告诉服务模型工件位置。
  - `"--epochs=" + EPOCHS`：训练的epoch数。
  - `"--steps=" + STEPS`：每个epoch的步骤（批量）数。
  - `"--distribute=" + TRAIN_STRATEGY"`：用于单设备或分布式训练的培训分发策略。
     - `"single"`：单个设备。
     - `"mirror"`：单个计算实例上的所有GPU设备。
     - `"multi"`：所有计算实例上的所有GPU设备。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_boston.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 组装工作规格

现在组装完整的自定义工作规格描述：

- `display_name`：您分配给这个自定义工作的可读名称。
- `job_spec`: 自定义工作的规范。
    - `worker_pool_specs`: 机器VM实例的规范。
    - `base_output_directory`: 这告诉服务在哪里保存模型工件的云存储位置（当变量`DIRECT = False`时）。然后，服务将将位置传递给训练脚本作为环境变量`AIP_MODEL_DIR`，路径将采取以下形式：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 检查培训套件

#### 套件布局

在开始培训之前，您将查看一个Python套件是如何为定制培训作业进行组装的。当解压缩时，套件包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将套件安装到Docker镜像的操作环境中的说明。

文件`trainer/task.py`是执行定制培训作业的Python脚本。*注意*，当我们在工作池规范中引用它时，我们将目录斜杠替换为点（`trainer.task`）并丢弃文件后缀（`.py`）。

#### 套件组装

在接下来的单元中，您将组装培训套件。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 内容

在下一个单元格中，您将编写训练脚本 task.py 的内容。我不会详细介绍，只是让您浏览一下。总结如下：

- 从命令行获取要保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF.Keras 内置数据集加载波士顿房屋数据集。
- 使用 TF.Keras 模型 API 构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分布策略。
- 用参数 `args.epochs` 指定的 epochs 训练模型（`fit()`）。
- 将经过训练的模型保存到指定的模型目录（`save(args.model_dir)`）。
- 将每个特征的最大值 `f.write(str(params))` 保存到指定的参数文件。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的tar文件，然后将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_boston.tar.gz

### 训练模型

现在开始在Vertex上训练您的自定义训练作业。使用以下辅助函数`create_custom_job`，该函数接受以下参数：

-`custom_job`：自定义作业的规范。

辅助函数调用作业客户端服务的`create_custom_job`方法，参数如下：

-`parent`：指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
-`custom_job`：自定义作业的规范。

您将在`response`对象中显示部分返回的字段，其中最感兴趣的两个字段是：

`response.name`：分配给此自定义训练作业的Vertex完全限定标识符。您需要保存此标识符以便在后续步骤中使用。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

### 获取有关自定义作业的信息

接下来，使用这个辅助函数 `get_custom_job`，它接受以下参数：

- `name`：自定义作业的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的 `get_custom_job` 方法，并带有以下参数：

- `name`：自定义作业的 Vertex 完全限定标识符。

如果您记得，在调用 `create_custom_job` 方法时，您在 `response.name` 字段中获得了自定义作业的 Vertex 完全限定标识符，并将标识符保存在变量 `job_id` 中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

# 部署

训练上述模型可能需要超过20分钟的时间。

一旦您的模型训练完成，您可以通过减去`end_time`和`start_time`来计算训练模型实际花费的时间。对于您的模型，我们需要知道保存模型的位置，Python脚本将保存在您的本地云存储桶中的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

## 加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在一个 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶中加载它，然后您可以执行一些操作，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras 的 `model.load_model()` 方法，将保存模型的 Cloud Storage 路径传递给它 -- 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们找出模型的表现如何。

### 加载评估数据

您将从`tf.keras.datasets`中加载波士顿房屋测试（留出）数据，使用`load_data()`方法。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：特征数据和对应的标签（业主自住房屋的中位价值）。

您不需要训练数据，因此我们加载为`(_, _)`。

在将数据用于评估之前，您需要对其进行预处理：

x_test:
1. 将每列中的数据进行归一化（重新缩放），方法是将每个值除以该列的最大值。这将用一个介于0和1之间的32位浮点数替换每个单个值。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature


# Let's save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

### 进行模型评估

现在评估一下自定义任务中的模型表现如何。

In [ ]:
model.evaluate(x_test, y_test)

## 上传用于提供服务的模型

接下来，您将从自定义作业上传您的TF.Keras模型到Vertex `Model`服务，这将为您的自定义模型创建一个Vertex `Model`资源。在上传过程中，您需要定义一个提供服务的函数，将数据转换为模型期望的格式。如果您将编码数据发送到Vertex，您的提供服务函数将确保在将数据作为输入传递给您的模型之前，在模型服务器上将数据解码。

### 提供服务函数如何工作

当您向在线预测服务器发送请求时，请求将被HTTP服务器接收。HTTP服务器从HTTP请求内容体中提取出预测请求。提取出的预测请求将被转发到提供服务的函数。对于Google预构建的预测容器，请求内容将作为 `tf.string` 传递给提供服务的函数。

提供服务函数由两部分组成：

- `预处理函数`：
  - 将输入 (`tf.string`) 转换为底层模型 (动态图) 的输入形状和数据类型。
  - 执行与训练底层模型期间相同的数据预处理 -- 例如，标准化、缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序所期望的格式 -- 例如，压缩输出。
  - 封装输出供接收应用程序使用 -- 例如，添加标题，制作JSON对象等。

预处理和后处理函数都将转换为与模型融合的静态图。从底层模型的输出传递给后处理函数。后处理函数将转换/封装后的输出返回给HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在构建TF.Keras模型的提供服务函数时，您需要考虑一个问题，那就是它们作为静态图运行。这意味着您不能使用需要动态图的TF图操作。如果这样做，您将在提供服务函数的编译过程中出现错误，提示您正在使用不受支持的EagerTensor。

获取服务功能签名

您可以通过重新加载模型到内存，并查询每个层对应的签名来获取模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要知道服务功能的输入层名称--在稍后进行预测请求时将使用。

您还需要知道服务功能的输入和输出层名称来构建解释元数据--稍后将讨论。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

input_name = model.input.name
print("Model input name:", input_name)
output_name = model.output.name
print("Model output name:", output_name)

### 说明规范

要在进行预测时获得解释，您必须启用解释功能，并在将自定义模型上传到 Vertex `Model` 资源时设置相应的设置。这些设置被称为解释元数据，包括：

- `parameters`：这是用于解释模型的可解释性算法的规范。您可以选择以下选项之一：
  - Shapley - *注意*，不推荐用于图像数据 -- 运行时间可能很长
  - XRAI
  - Integrated Gradients
- `metadata`：这是算法在您的自定义模型上应用的规范。

#### 解释参数

让我们首先深入了解可解释性算法的设置。

#### Shapley

为每个特征的结果分配信用，并考虑不同的特征排列。该方法提供了精确 Shapley 值的抽样近似。

用例：
  - 在表格数据上进行分类和回归。

参数：

- `path_count`：这是算法处理的特征路径数。对精确的 Shapley 值需要M! 个路径，其中 M 是特征数量。对于 CIFAR10 数据集，这将是 784 (28*28)。

对于任何非平凡数量的特征，这太耗费计算资源了。您可以将特征路径的数量减少到 M * `path_count`。

#### Integrated Gradients

一种基于梯度的方法，用于有效计算具有与 Shapley 值相同公理属性的特征属性。

用例：
  - 在表格数据上进行分类和回归。
  - 在图像数据上进行分类。

参数：

- `step_count`：这是用于近似其余总和的步骤数。步骤数越多，积分近似就越精确。一般的经验法则是 50 步，但随着步数的增加，计算时间也会增加。

#### XRAI

基于综合梯度方法，XRAI 评估图像的重叠区域，创建一个显着性地图，突出图像的相关区域而不是像素。

用例：

  - 在图像数据上进行分类。

参数：

- `step_count`：这是用于近似其余总和的步骤数。步骤数越多，积分近似就越精确。一般的经验法则是 50 步，但随着步数的增加，计算时间也会增加。

在下一个代码单元格中，将变量 `XAI` 设置为您将在自定义模型上使用的解释算法。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aip.ExplanationParameters(PARAMETERS)

#### 解释元数据

让我们首先深入了解解释元数据，其中包括：

- `outputs`：输出中的标量值属性--要解释的内容。例如，在分类中，对于概率输出\[0.1, 0.2, 0.7\]，我们想要解释0.7。考虑以下公式，其中输出是 `y`，这是我们想要解释的内容。

    y = f(x)

考虑到下面的公式，输出是 `y` 和 `z`。因为我们只能对一个标量值进行归因，所以我们必须选择是要解释输出 `y` 还是 `z`。假设在这个例子中模型是目标检测，y 和 z 分别是边界框和物体分类。你需要选择要解释的这两个输出中的哪一个。

    y, z = f(x)

`outputs` 的字典格式为：

    { "outputs": { "[your_display_name]":
                   "output_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]：你为要解释的输出赋予的人类可读名称。一个常见的例子是 "概率"。<br/>
 -  "output_tensor_name"：用于识别要解释的输出层的键/值字段。<br/>
 -  [layer]：要解释的输出层。在单任务模型中，如表格回归器，它是模型中的最后（最顶部）一层。
</blockquote>

- `inputs`：属性的特征--它们如何影响输出。考虑以下公式，其中 `a` 和 `b` 是特征。我们必须选择哪些特征来解释它们如何影响输出。假设这个模型用于A/B测试，其中 `a` 是用于预测的数据项，`b` 识别模型实例是 A 还是 B。你想选择 `a`（或一些子集）作为特征，而不选择 `b`，因为它并不会对预测产生影响。

    y = f(a,b)

`inputs` 的最小字典格式为：

    { "inputs": { "[your_display_name]":
                  "input_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]: 你为要解释的输入分配的人类可读名称。一个常见的例子是 "特征"。<br/>
 -  "input_tensor_name": 用于标识特征归因输入层的键/值字段。<br/>
 -  [layer]: 用于特征归因的输入层。在单输入张量模型中，它是模型中的第一个（最底部）层。
</blockquote>

由于模型的输入是表格形式的，你可以指定以下两个额外的字段作为报告/可视化辅助：

<blockquote>
 - "modality": "image"：表示字段值是图像数据。
</blockquote>

由于模型的输入是表格形式的，你可以指定以下两个额外的字段作为报告/可视化辅助：

<blockquote>
 - "encoding": "BAG_OF_FEATURES"：表示输入是一组表格特征。<br/>
 - "index_feature_mapping": [特征名称]：每个特征的人类可读名称列表。在这个例子中，我们使用数据集中指定的特征名称。<br/>
 - "modality": "numeric"：表示字段值是数值的。
</blockquote>

In [ ]:
INPUT_METADATA = {
    "input_tensor_name": serving_input,
    "encoding": "BAG_OF_FEATURES",
    "modality": "numeric",
    "index_feature_mapping": [
        "crim",
        "zn",
        "indus",
        "chas",
        "nox",
        "rm",
        "age",
        "dis",
        "rad",
        "tax",
        "ptratio",
        "b",
        "lstat",
    ],
}

OUTPUT_METADATA = {"output_tensor_name": serving_output}

input_metadata = aip.ExplanationMetadata.InputMetadata(INPUT_METADATA)
output_metadata = aip.ExplanationMetadata.OutputMetadata(OUTPUT_METADATA)

metadata = aip.ExplanationMetadata(
    inputs={"features": input_metadata}, outputs={"medv": output_metadata}
)

explanation_spec = aip.ExplanationSpec(metadata=metadata, parameters=parameters)

### 上传模型

使用这个辅助函数`upload_model`来上传您的模型，存储在SavedModel格式中，到`Model`服务，该服务将为您的模型实例化一个Vertex`Model`资源实例。一旦您完成了这一步，您就可以像任何其他Vertex`Model`资源实例一样使用`Model`资源实例，比如部署到用于提供预测的`Endpoint`资源。

该辅助函数接受以下参数：

- `display_name`：`Endpoint`服务的可读名称。
- `image_uri`：用于模型部署的容器镜像。
- `model_uri`：SavedModel工件的云存储路径。对于本教程，这是`trainer/task.py`保存模型工件的云存储位置，我们在变量`MODEL_DIR`中指定。

该辅助函数调用`Model`客户端服务的`upload_model`方法，该方法接受以下参数：

- `parent`：`Dataset`、`Model`和`Endpoint`资源的Vertex根路径。
- `model`：Vertex`Model`资源实例的规范。

现在让我们更深入地了解Vertex模型规范`model`。这是一个包含以下字段的字典对象：

- `display_name`：`Model`资源的可读名称。
- `metadata_schema_uri`：由于您的模型是没有使用Vertex`Dataset`资源构建的，您可以将其留空（`''`）。
- `artificat_uri`：以SavedModel格式存储模型的云存储路径。
- `container_spec`：这是将安装在`Endpoint`资源上的Docker容器的规范，从该容器，`Model`资源将提供预测。使用您之前设置的变量`DEPLOY_GPU！= None`来使用GPU；否则只分配一个CPU。
- `explanation_spec`：这是为您的模型启用可解释性的规范。

上传模型到Vertex模型资源将返回一个长时间操作，因为可能需要一些时间。您可以调用response.result()，这是一个同步调用，当Vertex模型资源准备好时将返回。

辅助函数返回对应的Vertex模型实例upload_model_response.model的Vertex完全限定标识符。您将保存该标识符以进行后续步骤中的变量model_to_deploy_id。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):

    model = aip.Model(
        display_name=display_name,
        artifact_uri=model_uri,
        metadata_schema_uri="",
        explanation_spec=explanation_spec,
        container_spec={"image_uri": image_uri},
    )

    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "boston-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy
)

获取`Model`资源信息

现在让我们获取只有您的模型的模型信息。使用这个帮助函数`get_model`，使用以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

这个帮助函数调用Vertex `Model`客户端服务的方法`get_model`，使用以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

## 部署 `Model` 资源

现在部署已训练的 Vertex 自定义 `Model` 资源。 这需要两个步骤：

1. 为部署 `Model` 资源创建一个 `Endpoint` 资源。

2. 将 `Model` 资源部署到 `Endpoint` 资源。

### 创建一个 `Endpoint` 资源

使用这个辅助函数 `create_endpoint` 来创建一个用于部署模型以提供预测的终端节点，参数如下：

- `display_name`: 用于 `Endpoint` 资源的可读性较强的名称。

这个辅助函数使用终端节点客户端服务的 `create_endpoint` 方法，该方法接受以下参数：

- `display_name`: 用于 `Endpoint` 资源的可读性较强的名称。

创建一个 `Endpoint` 资源会返回一个长时间运行的操作，因为需要一些时间来为服务准备 `Endpoint` 资源。您需要调用 `response.result()`，这是一个同步调用，在 `Endpoint` 资源准备好后会返回。辅助函数返回 `Endpoint` 资源的 Vertex 完整标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "boston_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获取您创建的“Endpoint”资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例扩展

处理在线预测请求时，您可以选择多种方式来扩展计算实例：

- 单个实例：在线预测请求在单个计算实例上进行处理。
  - 将计算实例的最小 (`MIN_NODES`) 和最大 (`MAX_NODES`) 数量设置为一。

- 手动扩展：在线预测请求被分配到您手动指定的固定数量的计算实例上。
  - 将计算实例的最小 (`MIN_NODES`) 和最大 (`MAX_NODES`) 数量设置为相同数量的节点。当模型首次部署到实例中时，会提供固定数量的计算实例，并且在线预测请求会均匀分布在这些实例上。

- 自动扩展：在线预测请求被分配到可扩展数量的计算实例上。
  - 在模型首次部署时，将计算实例的最小 (`MIN_NODES`) 数量设置为提供并且取消提供的实例数量，将最大 (`MAX_NODES`) 数量设置为根据负载情况提供的计算实例数量。

计算实例的最小数量对应字段 `min_replica_count`，最大数量对应字段 `max_replica_count`，在您之后的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 部署 `Model` 资源到 `Endpoint` 资源

使用此辅助函数 `deploy_model` 将模型部署到为提供预测服务创建的端点，使用以下参数：

- `model`：要从训练管道上传（部署）的 `Model` 资源的 Vertex 全部限定标识符。
- `deploy_model_display_name`：部署模型的可读名称。
- `endpoint`：要将 `Model` 资源部署到的 Vertex 全部限定的 `Endpoint` 资源标识符。

辅助函数调用 `Endpoint` 客户服务的方法 `deploy_model`，该方法需要以下参数：

- `endpoint`：要将 `Model` 资源部署到的 Vertex 全部限定的 `Endpoint` 资源标识符。
- `deployed_model`：部署模型的要求。
- `traffic_split`：端点上流量的百分比将转到此模型，指定为一个或多个键值对的字典。
   - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 指的是要上传的这个模型，100 表示流量的 100%。
   - 如果已存在要分配流量的现有模型，则指定为以下格式，其中 `model_id` 是要部署到的端点的现有模型的模型 ID。百分比必须加起来为 100。

           { "0": 百分比, model_id: 百分比, ... }

现在让我们深入了解 `deployed_model` 参数。这个参数被指定为一个 Python 字典，至少包含以下各字段：

- `model`：要部署的 Vertex 全部限定标识符的（上传）`Model` 资源。
- `display_name`：部署模型的可读名称。
- `dedicated_resources`：指的是为提供预测请求而扩展的计算实例（副本）数量。
  - `machine_spec`：要提供的计算实例。使用您之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配 CPU。
  - `min_replica_count`：最初要提供的计算实例数目，您之前设置为变量 `MIN_NODES`。
  - `max_replica_count`：要扩展到的计算实例的最大数量，您之前设置为变量 `MAX_NODES`。
- `enable_container_logging`：这会启用容器事件的日志记录，例如执行失败（默认情况下容器日志记录被禁用）。容器日志记录通常在部署调试时启用，然后在生产环境部署时禁用。

#### 流量分配

现在让我们深入了解 `traffic_split` 参数。这个参数被指定为一个 Python 字典。这可能一开始有点令人困惑。让我解释一下，您可以将模型的多个实例部署到一个端点，并设置每个实例接收的百分比。

为什么这样做？也许您已经在生产环境中部署了先前的版本 -- 让我们称之为 v1。您在 v2 上获得了更好的模型评估，但在将其部署到生产环境之前，您并不确定它是否真的更好。因此，在流量分配的情况下，您可能希望将 v2 部署到与 v1 相同的端点，并仅让其获得 10% 的流量。这样，您可以在不影响大多数用户的情况下监视其表现，直到做出最终决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将同步等待操作完成，调用 `response.result()`，这将阻塞直到模型部署完毕。如果这是第一次将模型部署到端点，可能需要额外几分钟来完成资源的提供。

In [ ]:
DEPLOYED_NAME = "boston_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "enable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

## 通过可解释性进行在线预测请求

现在对您部署的模型进行在线预测，并附带解释性。在这种方法中，预测的响应将包括关于特征如何影响解释的说明。

获取测试项目

您将从数据集的测试（保留）部分中选择一个示例作为测试项目。

In [ ]:
test_item = x_test[0]
test_label = y_test[0]
print(test_item.shape)

### 发送带有解释请求的预测

好的，现在你有一个测试数据项。使用这个助手函数 `explain_data`，它接受以下参数：

- `data_item`：以浮点值的 numpy 1D 数组形式表示的测试数据项。
- `endpoint`：部署模型的 Vertex AI 完全限定标识符。
- `parameters_dict`：用于推理的额外参数。

这个函数使用预测客户端服务，并使用以下参数调用 `explain` 方法：

- `endpoint`：部署模型的 Vertex AI 完全限定标识符。
- `instances`：要进行预测的实例（数据项）的列表。
- `parameters`：用于推理的额外参数。*注意*，自定义模型不支持额外参数。
- `deployed_model_id`：当在端点部署多个模型时，部署模型的 Vertex AI 完全限定标识符。否则，如果只部署一个模型，则可以设置为 `None`。

要将测试数据传递给预测服务，您必须按照以下步骤将其打包以便传输到服务二进制文件：

    1. 将数据项从 1D numpy 数组转换为 1D Python 列表。
    2. 将预测请求转换为序列化的 Google protobuf (`json_format.ParseDict()`)

预测请求中的每个实例都是以下形式的字典条目：

                        {input_name: content}

- `input_name`：基础模型的输入层的名称。
- `content`：作为 1D Python 列表的数据项。

由于 `explain()` 方法可以接受多个数据项（实例），您将把您的单个数据项发送为一个数据项的列表。最后一步是将实例列表打包成 Google 的 protobuf 格式--这就是我们传递给 `explain()` 方法的内容。

`response` 对象返回一个列表，列表中的每个元素对应请求中的对应图像。对于每个预测，您将会看到以下输出：

- `deployed_model_id` -- 进行预测/解释的模型的 Vertex AI 完全限定标识符。
- `predictions` -- 以 1K 美元为单位的房屋的预测中位数值。
- `explanations` -- 每个特征对预测的贡献方式。

In [ ]:
def explain_data(
    data_items, endpoint, parameters_dict, deployed_model_id, silent=False
):
    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = [
        json_format.ParseDict({serving_input: s.tolist()}, Value()) for s in data_items
    ]

    response = clients["prediction"].explain(
        endpoint=endpoint,
        instances=instances,
        parameters=parameters,
        deployed_model_id=deployed_model_id,
    )
    if silent:
        return response

    print("response")
    print(" deployed_model_id:", response.deployed_model_id)

    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)

    explanations = response.explanations
    print("explanations")
    for explanation in explanations:
        print(explanation)
    return response


response = explain_data([test_item], endpoint_id, None, None)

首先，让我们看一下您的模型预测的房屋中位价格，并将其与实际价值进行比较。

In [ ]:
predictions = response.predictions
print("Predicted Median House Value:", predictions[0][0], "1K USD")
print("Actual Mean House Value:", test_label, "1K USD")

### 检查特征归因

接下来，您将查看此特定示例的特征归因。正面贡献值意味着某个特征将您的模型预测提高了这么多，负面贡献值则表示相反。

In [ ]:
from tabulate import tabulate

feature_names = [
    "crim",
    "zn",
    "indus",
    "chas",
    "nox",
    "rm",
    "age",
    "dis",
    "rad",
    "tax",
    "ptratio",
    "b",
    "lstat",
]
attributions = response.explanations[0].attributions[0].feature_attributions

rows = []
for i, val in enumerate(feature_names):
    rows.append([val, test_item[i], attributions[val]])
print(tabulate(rows, headers=["Feature name", "Feature value", "Attribution value"]))

### 检查你的解释和基线

为了更好地理解你得到的特征归因，你应该将它们与模型的基线进行比较。在大多数情况下，你的归因值之和加上基线应该接近于模型对于每个输入的预测值。另外请注意，对于回归模型，从AI解释返回的`baseline_score`对于发送给模型的每个示例都是相同的。对于分类模型，每个类别将有自己的基线。

在这个部分，你将发送10个测试示例给你的模型进行预测，以便将特征归因与基线进行比较。然后你将通过`sanity_check_explanations`方法来检查每个测试示例的归因。

#### 获取解释

In [ ]:
# Prepare 10 test examples to your model for prediction
pred_batch = []
for i in range(10):
    pred_batch.append(x_test[i])

response = explain_data(pred_batch, endpoint_id, None, None, silent=True)

### 理智检查

在下面的函数中，您对解释进行了理智检查。

In [ ]:
def sanity_check_explanations(
    explanation, prediction, mean_tgt_value=None, variance_tgt_value=None
):
    passed_test = 0
    total_test = 1
    # `attributions` is a dict where keys are the feature names
    # and values are the feature attributions for each feature
    baseline_score = explanation.attributions[0].baseline_output_value
    print("baseline:", baseline_score)

    # Sanity check 1
    # The prediction at the input is equal to that at the baseline.
    #  Please use a different baseline. Some suggestions are: random input, training
    #  set mean.
    if abs(prediction - baseline_score) <= 0.05:
        print("Warning: example score and baseline score are too close.")
        print("You might not get attributions.")
    else:
        passed_test += 1
        print("Sanity Check 1: Passed")

    print(passed_test, " out of ", total_test, " sanity checks passed.")


i = 0
for explanation in response.explanations:
    try:
        prediction = np.max(response.predictions[i]["scores"])
    except TypeError:
        prediction = np.max(response.predictions[i])
    sanity_check_explanations(explanation, prediction)
    i += 1

取消部署 `Model` 资源

现在从提供服务的 `Endpoint` 资源中取消部署您的 `Model` 资源。使用这个辅助函数 `undeploy_model`，它接受以下参数：

- `deployed_model_id`：当 `Model` 资源部署到时，端点服务返回的模型部署标识符。
- `endpoint`：`Model` 部署到的 `Endpoint` 资源的 Vertex 完全限定标识符。

此函数调用端点客户端服务的 `undeploy_model` 方法，带有以下参数：

- `deployed_model_id`：端点服务在 `Model` 资源部署时返回的模型部署标识符。
- `endpoint`：`Model` 资源部署到的 `Endpoint` 资源的 Vertex 完全限定标识符。
- `traffic_split`：如何在 `Endpoint` 资源上的剩余部署模型之间分配流量。

由于这是 `Endpoint` 资源上唯一部署的模型，您可以直接通过将它设置为空对象 {} 来将 `traffic_split` 留空。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

整理清理

为了清理该项目中使用的所有GCP资源，您可以删除用于此教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 定制作业
- 超参数调整作业
- 云存储存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME